Importamos las librerías (obviamente voy a ir añadiendo librerías en base vaya creciendo el código)

In [2]:
import os
import torch
import shutil
from torch import nn
from torch.nn import functional as f
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image, UnidentifiedImageError
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv(r'..\..\labels\labels.csv')
df

,filename,label
0,10538.jpg,Cat
1,10539.jpg,Cat
2,1054.jpg,Cat
3,10540.jpg,Cat
4,10541.jpg,Cat
...,...,...
24760,8588.jpg,Dog
24761,9584.jpg,Dog
24762,1.jpg,Dog
24763,7968.jpg,Dog


El dataset en sí mismo tiene "Filename collision" gracias a explorar las carpetas de imágenes, pero bueno ¿Qué quiere decir este término? Bueno, el término **Filename collision** refiere a que distintas imágenes tiene el mismo nombre, en este caso accederemos a la imagen "0.jpg", lo normal sería que solo diera un resultado, pero como podremos ver, no es el caso.

In [4]:
df[df['filename'] == '0.jpg']

,filename,label
11803,0.jpg,Cat
12402,0.jpg,Dog


Como podemos ver, distintas imágenes tienen el mismo nombre '0.jpg', lo que puede causar problemas a la hora de entrenar el modelo, por ejemplo, si en el dataframe carga el archivo 0.jpg, Cat, pero el modelo (el DataLoader) carga 0.jpg, Dog, puede haber mala generalización, confusiones del modelo y resultados aleatorios.

Para resolver esto, pensé en renombrar los archivos en una misma carpeta, pero no perdería los datos, porque me basaré en sus índices para crear sus nuevos nombres, sin perder la etiqueta que son.

Primero realizamos el reseteo de índices del DataFrame con el parámetro drop en True para evitar que el índice que tiene, se convierta en una columna

In [5]:
df = df.reset_index(drop=True)

"""Una vez tenemos los índices, podemos crear los nombres, para esto creamos una columna/campo llamado "filename_nuevo" que su valor es el método index 
del DataFrame para obtener su índice y usamos .astype('str') para obtenerlo como string y a eso le añadimos el tipo de archivo, en este caso JPG"""

df['filename_nuevo'] = df.index.astype(str) + '.jpg'
df

,filename,label,filename_nuevo
0,10538.jpg,Cat,0.jpg
1,10539.jpg,Cat,1.jpg
2,1054.jpg,Cat,2.jpg
3,10540.jpg,Cat,3.jpg
4,10541.jpg,Cat,4.jpg
...,...,...,...
24760,8588.jpg,Dog,24760.jpg
24761,9584.jpg,Dog,24761.jpg
24762,1.jpg,Dog,24762.jpg
24763,7968.jpg,Dog,24763.jpg


Como podemos observar, ya tenemos los datos con un solo nombre, evitando esto el problema "Filename collision", ahora debemos copiarlo a la carpeta donde estarán todos los archivos, esto se hace con la librería os para los paths y shutil para mover los archivos.

Primero creamos la ruta base donde se encuentran todas las carpetas y sus imágenes: ..\ ..\PetImages

Después debemos iterar sobre las filas del DataFrame, lo cual podemos realizarlo con el método iterrows() del DataFrame y un bucle for con tuplas (idx, fila)

In [6]:
ruta_base = r'..\..\PetImages'

for idx, fila in df.iterrows():
    #Ahora debemos obtener el label, el anterior nombre y el nuevo nombre que cree con sus índices
    label = fila['label']
    old_name = fila['filename']
    new_name = fila['filename_nuevo']

    #Ahora con la anterior información podemos crear los paths, donde imaginemos que label = Dog, old_name='0.jpg' y el nuevo nombre es 10.jpg
    old_path = os.path.join(ruta_base, label, old_name) # Resultado: C:\Users\PC\Desktop\Road_to_Hackathon\PetImages\Dog\0.jpg
    dir_path = os.path.join(ruta_base, 'renamed_ds')  # Resultado: C:\Users\PC\Desktop\Road_to_Hackathon\PetImages\renamed_ds (Esta carpeta la llame así porque me quedé sin ideas jaja)
    new_path = os.path.join(ruta_base, 'renamed_ds', new_name) # Resultado: C:\Users\PC\Desktop\Road_to_Hackathon\PetImages\renamed_ds\10.jpg
    
    #Ahora necesitamos hacer una condición para saber si es que existe o no el directorio renamed_ds (esto solo pa' la primera vez que ejecuto el código jajajsfaj)
    if not os.path.isdir(dir_path): # 1era ejecución: False -1: True lmao xd
        #Ahora creamos el directorio
        os.mkdir(dir_path)
    
    #Ahora copiamos el archivo
    shutil.copy(old_path, new_path)

print("Los archivos han sido copiados")


Los archivos han sido copiados


![Imagen de la carpeta renamed_ds](../../Images/Random/folder_renamedDS.png)

Ala, aparcao' ya que tenemos todos los archivos movidos, solo queda limpiar el dataframe para pasarlo a el slice de dataset para entrenamiento y validación, solo para corroborar que no haya Filename collision ejecutamos el siguiente bloque de código:

In [7]:
df[df['filename_nuevo'] == '0.jpg']

,filename,label,filename_nuevo
0,10538.jpg,Cat,0.jpg


Como podemos observar todavía debemos borrar la columna que podría llegar a colisiones de nombre, pero tomando como argumento la columna del DataFrame "filename_nuevo"

In [8]:
#Ahora borramos la columna 'filename' que tiene el problema del Filename collision, con el parámetro: inplace con el valor de True, para que modifique el DataFrame y no cree una copia.
df.drop(columns=['filename'], inplace=True)

In [9]:
#Renombramos la columna filename_nuevo a filename
df.rename(columns={"filename_nuevo":'filename'}, inplace=True)

In [10]:
#Creamos una lista con los nombres de las columnas del DataFrame
columnas = df.columns.tolist()

#Quitamos el campo/columna filename
columnas.remove('filename')

In [11]:
#Nomás pa' ver las columnas en la lista XD
columnas

['label']

In [12]:
#Insertamos en la posición 0 el argumento 'filename'
columnas.insert(0, 'filename')

In [13]:
#Volvemos a mostrar, ala aparcao' de nuevo pa' poder ponerlas en el DataFrame
columnas

['filename', 'label']

In [14]:
#Renombramos las columnas del DataFrame y lo mostramos pa' corroborar
df = df[columnas]
df

,filename,label
0,0.jpg,Cat
1,1.jpg,Cat
2,2.jpg,Cat
3,3.jpg,Cat
4,4.jpg,Cat
...,...,...
24760,24760.jpg,Dog
24761,24761.jpg,Dog
24762,24762.jpg,Dog
24763,24763.jpg,Dog


In [15]:
#Ahora si ejecutamos el método que ya teníamos anteriormente, pero con el argumento filename porque ya renombramos el.
df[df['filename'] == '0.jpg']

,filename,label
0,0.jpg,Cat


Ahora que tenemos el DataFrame, podemos crear la clase para crear el Dataset customizado para el modelo que entrenaremos. 

El código es similar al que nos otorga la documentación de PyTorch, donde como instancias declaramos 4 la cual es el **DataFrame**, el **directorio de imágenes**, **transformador de imágenes** (normalizaciones, convertirlo a tensor de PyTorch, DataAugmentation, etc.) y **transformador de etiquetas** (convertir de one-hot a label encoding o de label encoding a logits [probabilidades])

In [16]:

class CustomDS(Dataset):
    def __init__(self, dataframe, images_dir, transform=None, target_transform=None): #Parámetros modificables
        self.labels = dataframe #Le pasamos el DataFrame que ya habíamos hecho sin el problema de Filename collision
        self.directory = images_dir #Ruta al directorio de imágenes
        self.transform_data = transform #Transformador de la imagen
        self.target_transform = target_transform #Transformador de etiquetas
        self.mapping = {"Cat": 0, "Dog": 1} #Mapeo de etiquetas

    def __len__(self):
        return len(self.labels) #Solamente obtiene la cantidad de elementos

    def __getitem__(self, idx):
        label_df = self.labels.iloc[idx, 1] #Obtiene el elemento de la fila (supongamos idx = 0) 0 y columna 1, lo que sería el elemento de la posición [0, 1] si vieramos al DataFrame como una matriz
        img_path = os.path.join(self.directory, self.labels.iloc[idx, 0]) #Generamos el path de donde se encuentran las imágenes (Notese que solo accedo al elemento, porque ya no hay distinción entre imágenes hablando de las carpetas)
        img = Image.open(img_path).convert('RGB') #Abrimos la imagen con PIL para saber si la imagen no está corrupta o dañada y si esto es cierto, la convertimos a RGB para tener 3 canales en el tensor
        img_label = self.mapping[label_df] #Mapeamos la etiqueta, es decir, si la etiqueta es Cat, la convierte en 0 o si es Dog la convierte en 1
        if self.transform_data: #Revisamos si se pasaron transformadores para los datos
            img = self.transform_data(img)
        if self.target_transform: #Lo mismo, pero para etiquetado
            img_label = self.target_transform(img_label)
        return img, img_label #Retornamos la imagen y su etiqueta

Solo quiero mencionar dos cosas que no mencioné en el código anterior, el código anterior en la sección de "transformador de etiquetas" me refiero al hecho que la etiqueta, puede permanecer como un entero o como un tensor donde ese tensor tiene el formato de **One-hot encoding** ¿A qué hace referencia este término? Es simplemente que si tenemos por ejemplo... la etiqueta 0, este número entero se convierte a un tensor donde se marca la posición 0 del tensor como un 1 y la demás 0 para determinar que es un 0, algo más gráfico sería:

0 -> torch.nn.functional.one_hot(torch.Tensor(label), num_classes=2) -> tensor([1, 0]) //La posición 0 del tensor marcado por el número 1, es el que determina a qué clase pertenece.

Además el otro punto que quería mencionar es que al momento en el que se cargan los archivos, es donde el problema de **Filename collision** es el que podría generar el problema de carga distinta imagen a la que el etiquetado sugiere. Para ser más claros pondré un ejemplo:

Imaginemos que el modelo carga la imagen 0.jpg de la etiqueta "Gato", pero el modelo carga la otra imagen de la etiqueta "Perro" lo que causa que el modelo aprenda al revés el generalizar las imágenes

In [17]:
#Ahora debemos dividir el DataFrame con train_test_split de Scikit-learn (Opté por usar dicho método de Scikit porque ya esta optimizado y además es más sencillo de usar [no debo reinventar la rueda])
#con 20% de los datos para el dataset de validación y 80% para entrenar el modelo, además dividiremos el dataset de validación, para obtener una métrica más confiable con imágenes que nunca ha visto

train_df, val_df = train_test_split(
    df,
    test_size = 0.2,
    stratify=df['label'],
    random_state=788
)

val_df, dev_df = train_test_split(
    val_df,
    test_size=0.1,
    stratify=val_df['label'],
    random_state=788
)

Una vez tenemos los DataFrames de entrenamiento, validación y prueba / dev (asi lo llaman coloquialmente xd), podemos ahora si crear una instancia de cada uno de los 3 datasets para el modelo, pero antes revisemos los tamaños de todos ellos.

In [18]:
general_path = r'C:\Users\PC\Desktop\Road_to_Hackathon\PetImages\renamed_ds'

print(f"Train size: {len(train_df)}, Test size: {len(val_df)}, Dev size: {len(dev_df)}")

transformador = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#Generación de las instancias de las clases CustomDS
train_dataset = CustomDS(dataframe=train_df, images_dir=general_path, transform=transformador)
val_dataset = CustomDS(dataframe=val_df, images_dir=general_path, transform=transformador)
dev_dataset = CustomDS(dataframe=dev_df, images_dir=general_path, transform=transformador)

#Generción de los DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=64, shuffle=True)

Train size: 19812, Test size: 4457, Dev size: 496


Una vez tenemos los datos (aún falta limpieza, pero ahorita más adelante lo resolvemos), generamos la arquitectura del modelo, pensé en el concepto de ResNet de usar dos convoluciones antes de pasar por un Pooling, solo que aquí opte por cortalo mediante un stride, algo que en el modelo final pienso cambiarlo por un MaxPooling porque para este problema solo debemos saber qué es, no debemos saber donde está y qué es (en ese caso stride = 2 está mejor que MaxPooling) ya que MaxPooling hace que se enfoque más en el objeto que más activaciones tuvo que en el que menos tuvo, filtrando algo de ruido de la imagen

In [19]:
#Aquí determinamos el dispositivo donde se harán todos los cálculos, en mi caso es una RTX 5070 TI TUF GAMING con 16 GB de VRam, pero optimizada para entrenamiento de modelos de Deep Learning por parte de NVIDIA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Generamos la clase del modelo con la herencia de nn.Module
class Modelo(nn.Module):
    def __init__(self):
        super(Modelo, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding = 1) #Input = 128, 128, 3
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3)) #63
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding = 1)
        self.conv4 = nn.Conv2d(in_channels= 32, out_channels=64, kernel_size=(3, 3)) #31
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding = 1)
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3)) #15
        self.conv7 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3)) #13
        self.conv8 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3)) #6
        self.maxpool = nn.MaxPool2d((2, 2))

        self.avg = nn.AdaptiveAvgPool2d((1, 1)) # batch, 128, 1, 1

        self.dense1 = nn.Linear(in_features=128, out_features=64)  
        self.dense2 = nn.Linear(in_features=64, out_features=2)

    def forward(self, x):
        x = f.relu(self.conv1(x))
        x = f.leaky_relu(self.maxpool(self.conv2(x)))
        x = f.relu(self.conv3(x))
        x = f.leaky_relu(self.maxpool(self.conv4(x)))
        x = f.relu(self.conv5(x))
        x = f.leaky_relu(self.maxpool(self.conv6(x)))
        x = f.relu(self.conv7(x))
        x = f.leaky_relu(self.maxpool(self.conv8(x)))

        x = self.avg(x)
        
        x = torch.flatten(x, 1)
        x = f.relu(self.dense1(x))
        x = self.dense2(x)

        return x

In [20]:
model = Modelo().to(device)
model.parameters

<bound method Module.parameters of Modelo(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (avg): AdaptiveAvgPool2d(output_size=(1, 1))
  (dense1): Linear(in_features=128, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=2, bias=True)
)>

In [25]:


function_loss = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(model.parameters(), lr=1e-4) 
"""
epochs = 100
best_loss = float('inf')
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0

    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = function_loss(outputs, labels)
        if loss < best_loss:
            best_model = loss
            torch.save(model.state_dict(), "Best_Model_Checkpoint.pth")
        optimizador.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) 
        optimizador.step()

    print(f'Época [{epoch+1}/{epochs}], loss: {loss.item():.4f}')

print('Entrenamiento finalizado.')
"""

'\nepochs = 100\nbest_loss = float(\'inf\')\nfor epoch in range(epochs):\n    model.train()\n    running_loss = 0.0\n    correct_preds = 0\n    total_samples = 0\n\n    for i, (images, labels) in enumerate(train_dataloader):\n        images = images.to(device)\n        labels = labels.to(device)\n\n        outputs = model(images)\n        loss = function_loss(outputs, labels)\n        if loss < best_loss:\n            best_model = loss\n            torch.save(model.state_dict(), "Best_Model_Checkpoint.pth")\n        optimizador.zero_grad()\n        loss.backward()\n        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) \n        optimizador.step()\n\n    print(f\'Época [{epoch+1}/{epochs}], loss: {loss.item():.4f}\')\n\nprint(\'Entrenamiento finalizado.\')\n'

In [22]:
#Ahora crearemos una función para limpiar datos corruptos o dañados usando la librería PIL con la clase Image y métodos open, para quitar el warning "UserWarning: Truncated File Read warnings.warn(str(msg))"
def clean_data(DirPath, DataFrame=None):
    DatosInaccesibles = []
    for _, row in DataFrame.iterrows():
        path = os.path.join(DirPath, row['filename'])
        try:
            with Image.open(path) as img:
                img.verify()
            with Image.open(path) as img:
                img.load()
        except (UnidentifiedImageError, OSError, ValueError) as ex:
            DatosInaccesibles.append(path)        
    return DatosInaccesibles

dictInaccesibles={}

ds_dir = r'..\..\PetImages'
df_dog = pd.read_csv(r'..\..\Labels\labels_Dog.csv')
df_cat = pd.read_csv(r'..\..\Labels\labels_Cat.csv')

print(df_dog, df_cat)

data_to_clean = [(os.path.join(ds_dir, 'Dog'), df_dog), (os.path.join(ds_dir, 'Cat'), df_cat)]

for path, dataframe in data_to_clean:
    print(path)
    dir_name = os.path.basename(path)
    dictInaccesibles[dir_name] = clean_data(DirPath=path, DataFrame=dataframe)

print(f"Archivos inaccesibles o dañados encontrados: {dictInaccesibles}")

       filename  label
0         0.jpg    Dog
1         1.jpg    Dog
2        10.jpg    Dog
3       100.jpg    Dog
4      1000.jpg    Dog
...         ...    ...
12358  9995.jpg    Dog
12359  9996.jpg    Dog
12360  9997.jpg    Dog
12361  9998.jpg    Dog
12362  9999.jpg    Dog

[12363 rows x 2 columns]        filename  label
0         0.jpg    Cat
1         1.jpg    Cat
2        10.jpg    Cat
3       100.jpg    Cat
4      1000.jpg    Cat
...         ...    ...
12397  9995.jpg    Cat
12398  9996.jpg    Cat
12399  9997.jpg    Cat
12400  9998.jpg    Cat
12401  9999.jpg    Cat

[12402 rows x 2 columns]
..\..\PetImages\Dog
..\..\PetImages\Cat
Archivos inaccesibles o dañados encontrados: {'Dog': [], 'Cat': []}


In [23]:
#Este fragmento lo hice para saber que imagen daba un warning y podría estar causando problemas de loss inestable
df_dog[df_dog['filename'] == '9040.jpg'].index

df_dog.iloc[11313, 0]

'9042.jpg'

In [39]:
class customDSAlone(Dataset):
    def __init__(self, dataframe, img_path, transform = None, target_transform = None):
        self.dataframe = dataframe
        self.img_dir = img_path
        self.transform = transform
        self.target_transform = target_transform
        self.mapping = {'Cat': 0, 'Dog':1}
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        label = self.dataframe.iloc[idx, 1]
        _filename = self.dataframe.iloc[idx, 0]
        path_img = os.path.join(self.img_dir, label, _filename)
        image = Image.open(path_img).convert('RGB')
        mapped_label = self.mapping[label]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, mapped_label, _filename

In [40]:
DogDF = pd.read_csv(r'..\..\Labels\labels_Dog.csv')
CatDF = pd.read_csv(r'..\..\Labels\labels_Cat.csv')
DogDS = customDSAlone(dataframe=DogDF, img_path=r'..\..\Petimages', transform=transformador)
CatDS = customDSAlone(dataframe=CatDF, img_path=r'..\..\Petimages', transform=transformador)
DogDL = DataLoader(dataset = DogDS, batch_size=64, shuffle=True)
CatDL = DataLoader(dataset = CatDS, batch_size=64, shuffle=True)

In [48]:
model = Modelo()
model.load_state_dict(torch.load('Best_Model_Checkpoint.pth', map_location=device))

def MismatchLabeling(model, dataloader, device, FunctionLoss):
    val_loss = 0.0
    total_samples = len(dataloader.dataset)
    correct = 0
    model.eval()
    model = model.to(device)
    resultados = []
    with torch.no_grad():
        for images, labels, filename in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images) #Aquí devuelve logits, por ejemplo si es solo 2 características (sería...) [2.1, 3.7]
            loss = FunctionLoss(outputs, labels) * images.size(0) #Acá se usa images.size(0) porque si tenemos batch = 32, pero con un loss 0.2, eso es el promedio, si quieres el general del batch, sería a la inversa, multiplicar por el total del batch
            val_loss += loss.item()
            prob = torch.nn.functional.softmax(outputs, 1)
            confianza, predict = torch.max(prob, 1)
            for i in range(images.size(0)):
                resultados.append({
                    'filename': filename[i],
                    'true_label': labels[i].item(),
                    'predicted_label': predict[i].item(),
                    'max_prob': confianza[i].item()
                })
                if labels[i] == predict[i]:
                    correct += 1
    val_loss /= len(dataloader.dataset)
    precision = (correct / total_samples) * 100
    return (f'El valor de pérdida del mdoelo es: {val_loss:.4f}, con una precisión de: {precision:.4f}', resultados)

In [49]:
Asd = MismatchLabeling(model = model, dataloader = DogDL, device = device, FunctionLoss = function_loss)
text, results = Asd

In [50]:
print(text)

El valor de pérdida del mdoelo es: 0.0856, con una precisión de: 97.0315


In [51]:
results

[{'filename': '9797.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.846744179725647},
 {'filename': '9165.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.999864935874939},
 {'filename': '1161.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.9845747947692871},
 {'filename': '3449.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.9992474317550659},
 {'filename': '936.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.99750816822052},
 {'filename': '6336.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.9856225848197937},
 {'filename': '3469.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.9937103986740112},
 {'filename': '8527.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.9003551602363586},
 {'filename': '6492.jpg',
  'true_label': 1,
  'predicted_label': 1,
  'max_prob': 0.9991887211799622},
 {'filename': '12442.jpg',
  'true_label': 1,
  'predicted_label': 1,